# Simplex Airfoils

Many indoor models are constructed using a simple circular arc airfoil. I showed how to use the Python **sympy** package to figure out the radius needed for some specified camber in my last article{cite}`black2021`.

The circular arc has its high point at the 50% chord point, which does not look much like a traditional airfoil. Another popular shape is called the *Simplex* airfoil, which moves the high point forward to about the 36% chord point. *Simplex* airfoils are unique in that one template can be used for any chord width by pinning the leading edge and rotating the template to match the desired chord. A bit of digging on the Internet turned up an Excel spreadsheet {cite}`simplex` that contained the code needed to generate the airfoil shape, but I want to be able to generate these shapes directly in Python. Let's extract the formulas needed from the spreadsheet and see what we get:

## Simplex Airfoil Spreadsheet Formulas

The formulas from the spreadsheet are these:

\begin{equation}
\alpha = 1.554 camber
\end{equation}

Here **camber** is the percentage of the chord desired.

\begin{equation}
\psi = \pi \tan(\frac{A \pi}{360})ln(x)
\end{equation}

Where $x$ is a number ranging from $0$ to $1$. This is not the save as the $are X$ coordinate we use for the airfoil. It seems to be related to $\rho/a$. We may need to figure this out later.

\begin{equation}
\rho = {\rm e}^{\psi/\tan(\frac{A\pi}{180})}
\end{equation}

\begin{equation}
x = \rho \sin{\psi}
\end{equation}

\begin{equation}
y = \rho \cos({-\psi})
\end{equation}

These equations work in generating the airfoil shape. However, I prefer to use verifiable equations that I can prove are correct, so we will explore generating the airfoil using **sympy** below.

For now, let's explore an example airfoil.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
chord = 2
camber = 0.10
npoints = 51
alpha = 1.554 * camber * math.pi/180
k = 1/math.tan(alpha)
print(alpha, k)

In [ ]:
roa = np.linspace(0,1,npoints)
roa = roa /chord
roa[0] = 0.0000001  # avoid infinity in ln(0)
psi = np.pi * np.tan(alpha/2) * np.log(roa)

In [ ]:
rho = chord * np.exp(psi / np.tan(alpha))
#rho[0] = 0
xc = rho * np.cos(psi)
yc = rho * np.sin(-psi)
print("  psi    rho     x      y")
for i in range(npoints):
    print("%6.3f %6.3f %6.3f %6.3f" % (psi[i], rho[i], xc[i], yc[i]))
    #print(psi[i], rho[i], xc[i], yc[i])

This looks exactly like the output from the spreadsheet. I used a hack to keep $r/a$ from reaching zero, which would have sent $ln(0r/a$ to infinity! There is still a "magic" number here that converts camber to the initial slope of the arc. I will try to figure that out later!

Let's get a look at this airfoil:

In [ ]:
plt.figure(figsize=(10,1))
plt.xlim(0, 1)
plt.ylim(0, 0.1)
plt.plot(xc,yc)
plt.show()

In [ ]:
mm = max(yc)
mmi = np.where(yc==mm)
print("Camber",max(yc),"at",xc[mmi][0])

It looks like this code will generate our airfoil shape. In order to generate aerodynamic coefficient data for this airfoil, we need to set up procedures that will return the camber line for this airfoil so it can be used by the airfoil generator code we will use to set up **xfoil** analysis.

Here is the procedure we will use:

In [ ]:
def simplex_camber_line(camber, npoints):
    """return list of points on camber line. camber is in percentage"""
    alpha = 1.554 * camber * math.pi/180
    k = 1/math.tan(alpha)
    
    # generate airfoil points
    roa = np.linspace(0,1,npoints)
    roa[0] = 0.0000001  # avoid infinity in ln(0)
    psi = np.pi * np.tan(alpha/2) * np.log(roa)
    rho = 1.0 * np.exp(psi / np.tan(alpha))
    x = rho * np.cos(psi)
    y = rho * np.sin(-psi)
    return x,y

In [ ]:
xc, yc = simplex_camber_line(4,51)    
plt.figure(figsize=(10,1))
plt.xlim(0, 1)
plt.ylim(0, 0.1)
plt.plot(xc,yc)
plt.show()

## Derivation of Simplex Airfoil Equations

This section is incomplete, and not necessary if you are willing to accept the code generated above. I am including it since I want to understand exactly how this airfoil gets generated.

The simplex airfoil is a section of the logarithmic spiral curve first developed in 1525 by  Albrecht Dürer and studied extensively by  Jacob Bernoulli, a name that should be familiar to anyone interested in aerodynamics!. This curve is defined by this equation:

\begin{equation}
\rho = a{\rm e}^{k\psi}
\end{equation}


In this equation, written in polar ($\rho$,$\psi$) coordinates, $a (a > 0)$ and $k (k \neq 0)$ are arbitrary constants.

From this defining equation, we can come up with some other helpful equations.

If we divide both sides by $a$ and take the natural logarithm of both sides, we get:

\begin{equation}
\psi = \frac{1}{k}ln\frac{\rho}{a}
\end{equation}


\begin{equation}
k = \frac{1}{\tan{\alpha}}
\end{equation}

Where the slope of the leading edge is defined by $\alpha$. 

We can define the equations for this curve in cartesian coordinates as follows:

\begin{equation}
x = a{\rm e}^{k\psi} \cos(\psi)
\end{equation}

\begin{equation}
y = a{\rm e}^{k\psi} \sin(\psi)
\end{equation}


As $k$ approaches zero, the spiral collapses to a circle. Looking at the documentation for this spiral, as $k$ approaches zero, the limiting case of $\alpha$ is $-\pi/2$. (I have not convinced **sympy** to prove this yet!) 

In [ ]:
import sympy
psi, a, k, rho = sympy.symbols('psi a k rho')

In [ ]:
eq1 = sympy.exp(k * psi)
eq2 = rho/a
eq3 = sympy.log(eq1)
eq4 = sympy.log(eq2)
print((eq3/k).expand(force=True),"=",eq4/k)

This is equation 7 above. 

The angle $\alpha$ between the tangent line to the spiral curve and a radial line is given by:
    
\begin{equation}
\tan(\alpha) = \frac{r}{\frac{dr}{d\psi}}
\end{equation}

In [ ]:
rho = a * sympy.exp(k * psi)
eq5= sympy.diff(rho, psi)
rho/eq5

This is equation 8 above. The angle is constant everywhere on the spiral. As $k \rightarrow 0$, the curve collapses to a simple circle with constant radius $a$.

In [ ]:
From this result, we get our equation for $\psi$ shown above,

It appears that the spreadsheet uses $\rho/a$ as a parameter, and lets it range from 0 to 1 (actually, from 0 to the chord length). Since the natural logarithm of zero is infinity, the value of $\psi$ is set to $-\pi/2$ when $\rho/a$ is equal to zero. This ends up with the $x$  value running from 0 to the chord length as well. This approximation cleans up the actual behavior of the spiral curve as $\rho/a$ approaches 0.

If we define the angle made between the spiral and the circle passing through this point as $\alpha$, we get:

\begin{equation}
tan(\alpha) = \frac{1}{k}
\end{equation}

Let's look at this curve:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n=100
a=0.25
k=0.20
th=np.linspace(0, 6*np.pi, 600)
x=a*np.exp(k*th)*np.cos(th)
y=a*np.exp(k*th)*np.sin(th)
plt.figure(figsize=(5,5))
plt.plot(x, y)
plt.plot([-10,10],[0,0])
plt.plot([0,0],[-10,10])
plt.show()

Obviously, we only need a slice of this curve for our airfoil. In this polar system, $psi$ is increasing in a counter-clockwise direction. At $\psi = 0$, the radius is simply equal to $a$. If you look closely at this spiral between angles of $0$ and $\pi/2$, it is apparent that as we sweep to increasing values of $\psi$ we are sweeping from the trailing edge of our desired airfoil to the leading edge. Let's set up a cartesian system that placed the origin at the leading edge $(\psi = 0)$ and the trailing edge at $(\psi = \pi/2)$. We will scale this new system so the length of the chord is one. 

### Slicing off the Simplex Airfoil

The spiral, as defined in polar form, cannot directly define our simplex airfoil. Instead, we need a coordinate transformation that sets up a new cartesian coordinate system we will use instead. 

### Locating the Maximum Point

We can explore these equations and try to locate the point where the height of our airfoil will be a maximum. 

In [ ]:
import sympy
y, k, psi = sympy.symbols('y k psi')
x = sympy.E**(k*psi) * sympy.sin(psi)
y = sympy.E**(k*psi) * sympy.cos(psi)

In [ ]:
e1 = sympy.diff(x,psi)
e1

In [ ]:
e2 = sympy.diff(y,psi)
e2

In [ ]:
sympy.ratsimp(e2/e1)

In the above equations $e1$ is $\frac{dx}{d\psi}$ and $e2$ is $\frac{dy}{d\psi}$. Using the chain rule, we get the slope of the curve $\frac{dy}{dx}$. The high point on the curve will occur where this expression equals zero. We can find the equation for $\psi$ where this happens:

In [ ]:
eq2 = k * sympy.cos(psi) - sympy.sin(psi)
eq2

In [ ]:
eq3 = sympy.solve(eq2,psi)
eq4 = sympy.simplify(eq3[0])
eq4

This is the value of $\psi$ where $y$ is at a max. To find out that height, we plug this back into the equation for $y$:

In [ ]:
plt.plot(x,y)
plt.show()

## Simplex Airfoil Function

In [ ]:
mm = slope(xp,yp)
plt.plot(xp,mm)
plt.plot([0,1],[0,0])
plt.show()

In [ ]:
def getLE(x0,y0,alpha):
    r0 = 0.005
    x = []
    y = []
    angles = np.linspace(np.pi/2 + alpha, 3*math.pi/2 + alpha, 19)
    for i in range(len(angles)):
        x.append(x0 + r0 * math.cos(angles[i]))
        y.append(y0 + r0 * math.sin(angles[i]))
    return x,y

In [ ]:
xl,yl = getLE(xa[-1], ya[-1],ma[-1])
f = plt.figure()
f.set_figwidth(5)
f.set_figheight(5)
plt.plot(xa,ma)
plt.plot(xl,yl)
plt.plot(xp,yp)
plt.plot(xa[100:],ya[100:])
plt.plot([0,0.02],[0.0,0.0])
plt.ylim(top=0.02, bottom = -0.02)
plt.xlim(left=-0.02,right=0.02)
plt.show()
print(ma[-1])

In [ ]:
def simplexCamber(x,y,m, thickness):
    xu = []
    yu = []
    xl = []
    yl = []
    for i in range(len(x)):
        xp = x[i]
        yp = y[i]
        angle = math.atan(m[i])
        xu.append(xp - thickness * math.sin(angle))
        yu.append(yp + thickness * math.cos(angle))
        xl.append(xp + thickness * math.sin(angle))
        yl.append(yp - thickness * math.cos(angle))
    return xu,yu,xl,yl

In [ ]:
xu,yu,xl,yl = simplexCamber(xa,ya,ma,0.005)
f = plt.figure()
f.set_figwidth(11)
f.set_figheight(2)
plt.ylim(top=0.15, bottom = -0.05)
plt.xlim(left=-0.05,right=1.05)
plt.plot(xu,yu)
plt.plot(xl,yl)
plt.plot(xp,yp)
plt.show()

In [ ]:
f = plt.figure()
f.set_figwidth(5)
f.set_figheight(5)
plt.plot(xa,ya)
plt.plot(xp,yp)
plt.plot(xu,yu)
plt.plot(xl,yl)
plt.plot(xa[100:],ya[100:])
plt.plot([0,0.02],[0.0,0.0])
plt.ylim(top=0.02, bottom = -0.02)
plt.xlim(left=-0.02,right=0.02)
plt.show()

In [ ]:
def getTE(xp, yp, mp, thickness):
    nx = len(xc)
    t = thickness/100
    r = t/2
    # set trailing edce distribution
    aa = r/25 - r
    cc = r
    xtopte = []
    ytopte = []
    xbotte = []
    ybotte = []

    for i in range(5+1):
        j = nx - 5 + i
        xx = i*0.2
        tt = aa*xx**2 + cc
        alpha = math.atan(mp[j]) + math.pi/2
        xc = xp[j]
        yc = yp[j]
        xt = xc + tt * math.cos(alpha)
        yt = yc + tt * math.sin(alpha)
        xtopte.append(xt)
        ytopte.append(yt)
        alpha += math.pi
        xb = xc + tt * math.cos(alpha)
        yb = yc + tt * math.sin(alpha)
        xbotte.append(xb)
        ybotte.append(yb)
    return xtopte, ytopte, xbotte, ybotte


